# DeepSTORM2D Add Ons
(1) Convert Picasso csv file to DeepSTORM2D csv format: <br/>
*Input file:* Picasso localization file csv, header = frame, x [nm], y [nm], sigma [nm], intensity [photon], offset [photon], bkgstd [photon], uncertainty_xy [nm] <br/>
*Output file:* DeepSTORM2D file csv, header = ,frame,x [nm],y [nm],Photon #,Sigma [nm]; frame idx starts at 1 <br/>

(2) Split tif movie & corresponding localization file at defined frame. <br/>
*Input file:* Tif movie & csv file (Picasso or DeepSTORM2D format). <br/>
*Parameter split at frame:* Number of frames of the first file. <br/>
*Output file:* Two tif movies & csv files, split at defined frame. <br/>

(3) Merge multiple movies & localization files. <br/>
*Input files:* Define multiple paths to movies and their localization files, adjust the number of files by deactivating "hide codecell" and adjusting the variable "n_measurements". Picasso or DeepSTORM2D csv formats are possible, but no mixing. <br/>
*Output file:* Tif movie with frames of movies stacked and localization file with continous frame numbering. <br/>

(4) Split into single frames. <br/>
*Input file:* Tif movie & csv file  (Picasso or DeepSTORM2D format). <br/>
*Output files:* Single frames of tif movies with corresponding csv file saved in defined directory. <br/>

In [1]:
from ImageBinner.widgets import widgetsAddOns, loadAddOns
from ImageBinner.tools import addOns
from ImageBinner.save import saveAddOns
import os
import warnings
warnings.filterwarnings("ignore")

## Convert localization file Picasso -> DeepSTORM2D format

### Load

In [2]:
loc_conv_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx="")
loc_conv_widget.file_button.on_click(loc_conv_widget.open_file)
loc_conv_widget.file_text_box.observe(loc_conv_widget.change_file_path_box)
display(loc_conv_widget.file_text_box, loc_conv_widget.file_button)

Text(value='', description='Path to localizations ', placeholder='Insert path', style=DescriptionStyle(descrip…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Convert

In [3]:
def run(event):
    widget_run.create_clear_output()
    display(widget_run.run_convert_button)
    if (loc_conv_widget.file_text_box.value):
        converter.convert(loc_conv_widget.file_text_box.value)
        display(converter.converted_file)
    else:
        print("Please define a file to convert in the loading section.")
        
converter = addOns.Converter()
widget_run = widgetsAddOns.RunAnalysis()
display(widget_run.run_convert_button)
widget_run.run_convert_button.on_click(run)

Button(description='convert', style=ButtonStyle(), tooltip='convert localization file')

### Save

In [4]:
def save_analysis(event):
    widget_conv_save.create_clear_output()
    display(widget_conv_save.save_button)
    saveAddOns.save_convert_file(loc_conv_widget.file_text_box.value, converter.converted_file)
    
widget_conv_save = widgetsAddOns.SaveResults()
display(widget_conv_save.save_button)
widget_conv_save.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Split movie & localization file at defined frame

### Load

In [5]:
movie_split_widget = widgetsAddOns.DefineFiles("movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx="")
loc_split_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx="")
movie_split_widget.file_button.on_click(movie_split_widget.open_file)
movie_split_widget.file_text_box.observe(movie_split_widget.change_file_path_box)
loc_split_widget.file_button.on_click(loc_split_widget.open_file)
loc_split_widget.file_text_box.observe(loc_split_widget.change_file_path_box)
display(movie_split_widget.file_text_box, movie_split_widget.file_button, loc_split_widget.file_text_box, loc_split_widget.file_button)

Text(value='', description='Path to movie ', placeholder='Insert path', style=DescriptionStyle(description_wid…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations ', placeholder='Insert path', style=DescriptionStyle(descrip…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Parameters

In [6]:
widget_split_params = widgetsAddOns.ParametersSplit()
display(widget_split_params.frame_idx)

Text(value='', description='Split at frame', placeholder='Insert frame idx', style=DescriptionStyle(descriptio…

### Split

In [7]:
def run(event):
    widget_split_run.create_clear_output()
    display(widget_split_run.run_split_button)
    data_split.load_data(movie_split_widget.file_text_box.value, loc_split_widget.file_text_box.value)
    if len(data_split.loc_file["frame"].unique()) != data_split.movie.shape[0]:
        print("Defined movie and localization file differ in length, please make sure that they correspond.")
    else:
        split.split(data_split.movie, data_split.loc_file, int(widget_split_params.frame_idx.value))
data_split = loadAddOns.DataSplit()
split = addOns.Splitter()
widget_split_run = widgetsAddOns.RunAnalysis()
display(widget_split_run.run_split_button)
widget_split_run.run_split_button.on_click(run)

Button(description='split', style=ButtonStyle(), tooltip='split data at frame')

### Save

In [8]:
def save_analysis(event):
    widget_split_save.create_clear_output()
    display(widget_split_save.save_button)
    saveAddOns.save_split_files(movie_split_widget.file_text_box.value, loc_split_widget.file_text_box.value, split.split_movies, split.split_locs, int(widget_split_params.frame_idx.value))

widget_split_save = widgetsAddOns.SaveResults()
display(widget_split_save.save_button)
widget_split_save.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Merge multiple movies & localization files

In [9]:
n_measurements = 2  # vary the number of measurements here!
movie_merge_widgets, loc_merge_widgets = [], []
for i in range(1, n_measurements+1):
    print("measurement", i)
    movie_widget = widgetsAddOns.DefineFiles("movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx=i)
    loc_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx=i)
    movie_widget.file_button.on_click(movie_widget.open_file)
    movie_widget.file_text_box.observe(movie_widget.change_file_path_box)
    loc_widget.file_button.on_click(loc_widget.open_file)
    loc_widget.file_text_box.observe(loc_widget.change_file_path_box)
    movie_merge_widgets.append(movie_widget)
    loc_merge_widgets.append(loc_widget)
    display(movie_widget.file_text_box, movie_widget.file_button, loc_widget.file_text_box, loc_widget.file_button)

measurement 1


Text(value='', description='Path to movie 1', placeholder='Insert path', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations 1', placeholder='Insert path', style=DescriptionStyle(descri…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

measurement 2


Text(value='', description='Path to movie 2', placeholder='Insert path', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations 2', placeholder='Insert path', style=DescriptionStyle(descri…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

In [10]:
def run(event):
    widget_merge_run.create_clear_output()
    display(widget_merge_run.run_merge_button)
    movie_dirs = [widget.file_text_box.value for widget in movie_merge_widgets if widget.file_text_box.value]
    loc_dirs = [widget.file_text_box.value for widget in loc_merge_widgets if widget.file_text_box.value]
    data_merge.load_data(movie_dirs, loc_dirs)
    shapes = [movie.shape[1:] for movie in data_merge.movies]
    if all(x==shapes[0] for x in shapes):
        merge.merge(data_merge.movies, data_merge.loc_files)
    else:
        print("All movies must be of equal width and height.") 

data_merge = loadAddOns.DataMerge()
merge = addOns.Merger()
widget_merge_run = widgetsAddOns.RunAnalysis()
display(widget_merge_run.run_merge_button)
widget_merge_run.run_merge_button.on_click(run)

Button(description='merge', style=ButtonStyle(), tooltip='merge files')

In [11]:
widget_merge_save = widgetsAddOns.SaveResults()
display(widget_merge_save.box_filename, widget_merge_save.dir_box, widget_merge_save.dir_button)
widget_merge_save.dir_button.on_click(widget_merge_save.open_dir)
widget_merge_save.dir_box.observe(widget_merge_save.change_dir_box)

Text(value='', description='Filename', placeholder='name of folder', style=DescriptionStyle(description_width=…

Text(value='', description='Directory', placeholder='Directory to save', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [12]:
display(widget_merge_save.save_button)
def save_merge(event):
    widget_merge_save.create_clear_output()
    display(widget_merge_save.save_button)   
    saveAddOns.save_merge_files(widget_merge_save.dir_box.value, widget_merge_save.box_filename.value, merge.merged_movies, merge.merged_locs)
widget_merge_save.save_button.on_click(save_merge)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Split into single frames

In [13]:
movie_single_split_widget = widgetsAddOns.DefineFiles("movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx="")
loc_single_split_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx="")
movie_single_split_widget.file_button.on_click(movie_single_split_widget.open_file)
movie_single_split_widget.file_text_box.observe(movie_single_split_widget.change_file_path_box)
loc_single_split_widget.file_button.on_click(loc_single_split_widget.open_file)
loc_single_split_widget.file_text_box.observe(loc_single_split_widget.change_file_path_box)
display(movie_single_split_widget.file_text_box, movie_single_split_widget.file_button, loc_single_split_widget.file_text_box, loc_single_split_widget.file_button)

Text(value='', description='Path to movie ', placeholder='Insert path', style=DescriptionStyle(description_wid…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations ', placeholder='Insert path', style=DescriptionStyle(descrip…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

In [14]:
def run_single_split(event):
    widget_single_split_run.create_clear_output()
    display(widget_single_split_run.run_single_split_button)
    data_single_split.load_data(movie_single_split_widget.file_text_box.value, loc_single_split_widget.file_text_box.value)
    if len(data_single_split.loc_file["frame"].unique()) != data_single_split.movie.shape[0]:
        print("Defined movie and localization file differ in length, please make sure that they correspond.")
    else:
        single_split.split(data_single_split.movie, data_single_split.loc_file)
        
data_single_split = loadAddOns.DataSplit()
single_split = addOns.SingleSplitter()
widget_single_split_run = widgetsAddOns.RunAnalysis()
display(widget_single_split_run.run_single_split_button)
widget_single_split_run.run_single_split_button.on_click(run_single_split)

Button(description='split', style=ButtonStyle(), tooltip='split data at every frame')

In [15]:
widget_single_split_save = widgetsAddOns.SaveResults()
display(widget_single_split_save.dir_box, widget_single_split_save.dir_button)
widget_single_split_save.dir_button.on_click(widget_single_split_save.open_dir)
widget_single_split_save.dir_box.observe(widget_single_split_save.change_dir_box)

Text(value='', description='Directory', placeholder='Directory to save', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [16]:
display(widget_single_split_save.save_button)
def save_single_split(event):
    widget_single_split_save.create_clear_output()
    display(widget_single_split_save.save_button)   
    saveAddOns.save_single_split_files(widget_single_split_save.dir_box.value, os.path.splitext(os.path.split(loc_single_split_widget.file_text_box.value)[1])[0], single_split.single_frame_movies, single_split.single_frame_locs)
widget_single_split_save.save_button.on_click(save_single_split)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

In [17]:
import numpy as np

x = np.zeros((2,2))
print(x)

[[0. 0.]
 [0. 0.]]
